In [21]:
import os
import datetime

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score
from tqdm import tqdm

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


import os
from pathlib import Path
import pandas as pd
import numpy as np

from scipy.signal import find_peaks
from scipy import signal


import pickle


path_aurora_dataset = Path('/Users/cslinxs/Desktop/aurora_dataset/aurora_dataset/')


def reject_outliers(data):
    #return data[abs(data - np.mean(data)) < 0.3 * np.mean(data)]
    return data[abs(data - np.mean(data)) <3 * np.std(data)]



dictionary_path = "/Users/cslinxs/Desktop/all_weeks/summer24-lab/measurements_auscultatory_dictionary_ALL.pkl"

with open(dictionary_path, 'rb') as handle:
    all_auscultatory_dic = pickle.load(handle)

# from utils import load_data, plot_history_torch, plot_heat_map


NameError: name 'pickle' is not defined

In [37]:
path_aurora_dataset = Path('/Users/cslinxs/Desktop/aurora_dataset/aurora_dataset/')
auscultatory_data = path_aurora_dataset / "measurements_auscultatory.tsv"

participants_data =  path_aurora_dataset / "participants.tsv"
basic_mesures_data = pd.read_csv(auscultatory_data,sep = '\t').dropna(subset=["waveform_file_path"]).dropna(subset=["sbp"]).dropna(subset=["dbp"]) 

def get_even_split(path):
    #print(path)
    df = pd.read_csv(path,sep = '\t')   
    pos_pressure = df["pressure"]

    neglist = [ -x for x in pos_pressure]

    b, a = signal.butter(1, 0.002, analog=False)
    gustafsson_avg = signal.filtfilt(b, a, x= pos_pressure, method="gust")
    y_flater = [real_pre-abs( guston_avg) for real_pre, guston_avg in zip(pos_pressure, gustafsson_avg)]

    y_flater = np.array(y_flater)
    neglist = [ -x for x in y_flater]

    neg_max_values = max(neglist)
    neg_mix_values = min(neglist)
    mid_neg_value= neg_max_values - neg_mix_values

    min_height = neg_max_values - mid_neg_value*0.30 

    min_peaks, _ = find_peaks(neglist , height= min_height, width=80)


    most_accurate_mins = min_peaks
    diffrences_new_min = reject_outliers(np.diff(most_accurate_mins))
    diffrences_new_min
    # plt.plot(y_flater)
    
    # plt.show()
    return  y_flater # (len(diffrences_new_min.mean()), pos_pressure)#len(diffrences_new_min.mean())
 

def prepare_data_frame_for_lstm(df,n_steps,pos_pressure ):
    df = df.pandas(df)
    for i in range(1, n_steps+1):
        df[f"shift \(t-{i})"] = pos_pressure.shift(i)


# everything[each_path] = [each_path_50_cords, filler_to_person[filler_to_person["waveform_file_path"] == each_path]["sbp"], 
#                          filler_to_person[filler_to_person["waveform_file_path"] == each_path]["dbp"] ]

        

In [38]:
all_data_recordings = []

for person in sorted(os.listdir(path_aurora_dataset/"measurements_auscultatory"))[:100]:
    if "a" in person:
        filler_to_person = basic_mesures_data[basic_mesures_data["pid"] == person]
        print("hi")
        for each_path in filler_to_person["waveform_file_path"]:
            if "Exercise" in each_path:
                print(each_path)
                flatlined_data = get_even_split(path_aurora_dataset/each_path)
                all_data_recordings.append(flatlined_data)

hi
measurements_auscultatory/a000/a000.initial.Exercise_challenge_start_1.tsv
measurements_auscultatory/a000/a000.initial.Exercise_challenge_start_2.tsv
hi
measurements_auscultatory/a001/a001.initial.Exercise_challenge_start_1.tsv
measurements_auscultatory/a001/a001.initial.Exercise_challenge_start_2.tsv
hi
measurements_auscultatory/a002/a002.initial.Exercise_challenge_start_1.tsv
hi
measurements_auscultatory/a003/a003.initial.Exercise_challenge_start_1.tsv
measurements_auscultatory/a003/a003.initial.Exercise_challenge_start_2.tsv
hi
measurements_auscultatory/a004/a004.initial.Exercise_challenge_start_1.tsv
measurements_auscultatory/a004/a004.initial.Exercise_challenge_start_2.tsv
hi
measurements_auscultatory/a005/a005.initial.Exercise_challenge_start_1.tsv
measurements_auscultatory/a005/a005.initial.Exercise_challenge_start_2.tsv
hi
measurements_auscultatory/a006/a006.initial.Exercise_challenge_start_1.tsv
measurements_auscultatory/a006/a006.initial.Exercise_challenge_start_2.tsv
hi
m

/Users/cslinxs/Library/Python/3.9/lib/python/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/cslinxs/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/cslinxs/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/cslinxs/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/cslinxs/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


hi
measurements_auscultatory/a051/a051.initial.Exercise_challenge_start_1.tsv
measurements_auscultatory/a051/a051.initial.Exercise_challenge_start_2.tsv
hi
measurements_auscultatory/a052/a052.initial.Exercise_challenge_start_1.tsv
measurements_auscultatory/a052/a052.initial.Exercise_challenge_start_2.tsv
hi
measurements_auscultatory/a053/a053.initial.Exercise_challenge_start_1.tsv
measurements_auscultatory/a053/a053.initial.Exercise_challenge_start_2.tsv
hi
measurements_auscultatory/a054/a054.initial.Exercise_challenge_start_1.tsv
measurements_auscultatory/a054/a054.initial.Exercise_challenge_start_2.tsv
hi
hi
hi
measurements_auscultatory/a057/a057.initial.Exercise_challenge_start_1.tsv
hi
measurements_auscultatory/a058/a058.initial.Exercise_challenge_start_1.tsv
measurements_auscultatory/a058/a058.initial.Exercise_challenge_start_2.tsv
hi
measurements_auscultatory/a059/a059.initial.Exercise_challenge_start_1.tsv
measurements_auscultatory/a059/a059.initial.Exercise_challenge_start_2.ts

In [35]:
print(all_data_recordings[:50])

[array([-4.81614166, -4.91633278, -5.016718  , ..., -8.04705951,
       -8.04642814, -7.94611344]), array([ 3.22835535,  3.02848083,  2.92872685, ..., -4.53197234,
       -4.63160368, -4.73141788]), array([-11.19155156, -11.39199536, -11.59288891, ..., -17.92033446,
       -18.11889793, -18.31817672]), array([  6.00406163,   5.80429669,   5.6047609 , ..., -33.01579379,
       -33.31315461, -33.61183061]), array([ 5.35659743,  4.95680496,  4.55720818, ..., -2.83950319,
       -2.9392721 , -2.93915607]), array([-11.35453872, -11.75499094, -12.15590723, ..., -35.51649867,
       -35.81366207, -36.11223935]), array([ 1.12953881,  4.72961894,  8.32988578, ..., 40.92741633,
       45.4236966 , 49.92177015]), array([ 76.14760558,  75.75060787,  75.45660171, ..., -20.90291742,
       -21.20123457, -21.50038871]), array([-27.15072646, -27.15179834, -27.15394216, ..., -12.2633743 ,
       -12.36239525, -12.46190425]), array([-46.18580714, -46.2876315 , -46.39128325, ..., -17.27158362,
       -17

In [39]:

# project root path
project_path = "./"
# define log directory
# must be a subdirectory of the directory specified when starting the web application
# it is recommended to use the date time as the subdirectory name
log_dir = project_path + "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
model_path = project_path + "ecg_model.pt"

# the device to use
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using {} device".format(device))


# define the dataset class
class ECGDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __getitem__(self, index):
        x = torch.tensor(self.x[index], dtype=torch.float32)
        y = torch.tensor(self.y[index], dtype=torch.long)
        return x, y

    def __len__(self):
        return len(self.x)


# build the CNN model
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        # the first convolution layer, 4 21x1 convolution kernels, output shape (batch_size, 4, 300)
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=4, kernel_size=21, stride=1, padding='same')
        # the first pooling layer, max pooling, pooling size=3 , stride=2, output shape (batch_size, 4, 150)
        self.pool1 = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        # the second convolution layer, 16 23x1 convolution kernels, output shape (batch_size, 16, 150)
        self.conv2 = nn.Conv1d(in_channels=4, out_channels=16, kernel_size=23, stride=1, padding='same')
        # the second pooling layer, max pooling, pooling size=3, stride=2, output shape (batch_size, 16, 75)
        self.pool2 = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        # the third convolution layer, 32 25x1 convolution kernels, output shape (batch_size, 32, 75)
        self.conv3 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=25, stride=1, padding='same')
        # the third pooling layer, average pooling, pooling size=3, stride=2, output shape (batch_size, 32, 38)
        self.pool3 = nn.AvgPool1d(kernel_size=3, stride=2, padding=1)
        # the fourth convolution layer, 64 27x1 convolution kernels, output shape (batch_size, 64, 38)
        self.conv4 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=27, stride=1, padding='same')
        # flatten layer, for the next fully connected layer, output shape (batch_size, 38*64)
        self.flatten = nn.Flatten()
        # fully connected layer, 128 nodes, output shape (batch_size, 128)
        self.fc1 = nn.Linear(64 * 38, 128)
        # Dropout layer, dropout rate = 0.2
        self.dropout = nn.Dropout(0.2)
        # fully connected layer, 5 nodes (number of classes), output shape (batch_size, 5)
        self.fc2 = nn.Linear(128, 5)

    def forward(self, x):
        # x.shape = (batch_size, 300)
        # reshape the tensor with shape (batch_size, 300) to (batch_size, 1, 300)
        x = x.reshape(-1, 1, 300)
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        x = F.relu(self.conv4(x))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


# define the training function and validation function
def train_steps(loop, model, criterion, optimizer):
    train_loss = []
    train_acc = []
    model.train()
    for step_index, (X, y) in loop:
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = criterion(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss = loss.item()
        train_loss.append(loss)
        pred_result = torch.argmax(pred, dim=1).detach().cpu().numpy()
        y = y.detach().cpu().numpy()
        acc = accuracy_score(y, pred_result)
        train_acc.append(acc)
        loop.set_postfix(loss=loss, acc=acc)
    return {"loss": np.mean(train_loss),
            "acc": np.mean(train_acc)}


def test_steps(loop, model, criterion):
    test_loss = []
    test_acc = []
    model.eval()
    with torch.no_grad():
        for step_index, (X, y) in loop:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = criterion(pred, y).item()

            test_loss.append(loss)
            pred_result = torch.argmax(pred, dim=1).detach().cpu().numpy()
            y = y.detach().cpu().numpy()
            acc = accuracy_score(y, pred_result)
            test_acc.append(acc)
            loop.set_postfix(loss=loss, acc=acc)
    return {"loss": np.mean(test_loss),
            "acc": np.mean(test_acc)}


def train_epochs(train_dataloader, test_dataloader, model, criterion, optimizer, config, writer):
    num_epochs = config['num_epochs']
    train_loss_ls = []
    train_loss_acc = []
    test_loss_ls = []
    test_loss_acc = []
    for epoch in range(num_epochs):
        train_loop = tqdm(enumerate(train_dataloader), total=len(train_dataloader))
        test_loop = tqdm(enumerate(test_dataloader), total=len(test_dataloader))
        train_loop.set_description(f'Epoch [{epoch + 1}/{num_epochs}]')
        test_loop.set_description(f'Epoch [{epoch + 1}/{num_epochs}]')

        train_metrix = train_steps(train_loop, model, criterion, optimizer)
        test_metrix = test_steps(test_loop, model, criterion)

        train_loss_ls.append(train_metrix['loss'])
        train_loss_acc.append(train_metrix['acc'])
        test_loss_ls.append(test_metrix['loss'])
        test_loss_acc.append(test_metrix['acc'])

        print(f'Epoch {epoch + 1}: '
              f'train loss: {train_metrix["loss"]}; '
              f'train acc: {train_metrix["acc"]}; ')
        print(f'Epoch {epoch + 1}: '
              f'test loss: {test_metrix["loss"]}; '
              f'test acc: {test_metrix["acc"]}')

        writer.add_scalar('train/loss', train_metrix['loss'], epoch)
        writer.add_scalar('train/accuracy', train_metrix['acc'], epoch)
        writer.add_scalar('validation/loss', test_metrix['loss'], epoch)
        writer.add_scalar('validation/accuracy', test_metrix['acc'], epoch)

    return {'train_loss': train_loss_ls,
            'train_acc': train_loss_acc,
            'test_loss': test_loss_ls,
            'test_acc': test_loss_acc}


def main():
    config = {
        'seed': 42,  # the random seed
        'test_ratio': 0.3,  # the ratio of the test set
        'num_epochs': 30,
        'batch_size': 128,
        'lr': 0.001,
    }

    # X_train,y_train is the training set
    # X_test,y_test is the test set

    auscultatory_pressure =[item[0] for item in all_auscultatory_dic.values()]
    auscultatory_sbp = [item[1] for item in all_auscultatory_dic.values()]
    X_train, X_test, y_train, y_test = train_test_split(auscultatory_pressure[:50],auscultatory_sbp[1][:50], test_size=0.3)
    train_dataset, test_dataset = ECGDataset(X_train, y_train), ECGDataset(X_test, y_test)
    train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False)

    # define the model
    model = Model()
    if os.path.exists(model_path):
        # import the pre-trained model if it exists
        print('Import the pre-trained model, skip the training process')
        model.load_state_dict(torch.load(model_path))
        model.eval()
    else:
        # build the CNN model
        model = model.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])

        # print the model structure
        summary(model, (config['batch_size'], X_train.shape[1]), col_names=["input_size", "kernel_size", "output_size"],
                verbose=2)

        # define the Tensorboard SummaryWriter
        writer = SummaryWriter(log_dir=log_dir)
        # train and evaluate model
        history = train_epochs(train_dataloader, test_dataloader, model, criterion, optimizer, config, writer)
        writer.close()
        # save the model
        torch.save(model.state_dict(), model_path)
        # plot the training history
        #plot_history_torch(history
        plt.plot(history)

    # predict the class of test data
    y_pred = []
    model.eval()
    with torch.no_grad():
        for step_index, (X, y) in enumerate(test_dataloader):
            X, y = X.to(device), y.to(device)
            pred = model(X)
            pred_result = torch.argmax(pred, dim=1).detach().cpu().numpy()
            y_pred.extend(pred_result)
    # plot confusion matrix heat map
    plt.plot(y_test, y_pred)


if __name__ == '__main__':
    main()

Using cpu device


NameError: name 'all_auscultatory_dic' is not defined